# Installing & loading packages

In [1]:
from fredapi import Fred
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.prompts.chat import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain.chains import LLMChain
import pandas as pd
import re
from datetime import datetime
from slack_sdk.errors import SlackApiError
from slack_sdk import WebClient
from dotenv import find_dotenv, load_dotenv
import os

# Load environment variables

In [2]:
load_dotenv(find_dotenv())

True

# Set Slack API credentials and FRED API key


In [3]:
SLACK_BOT_TOKEN = os.environ.get("SLACK_BOT_TOKEN_FRED")  # Using get to avoid KeyError
SLACK_SIGNING_SECRET = os.environ.get("SLACK_SIGNING_SECRET_FRED")
FRED_API_KEY = os.environ.get("FRED_API_KEY")
SLACK_BOT_USER_ID = os.environ.get("SLACK_BOT_USER_ID_FRED")

# Check if environment variables are loaded correctly

In [4]:
if not all([SLACK_BOT_TOKEN, SLACK_SIGNING_SECRET, FRED_API_KEY, SLACK_BOT_USER_ID]):
    raise ValueError("Missing environment variables. Please check your .env file.")


# Initialize the Slack app and Flask app

In [5]:
from flask import Flask, request
from slack_bolt import App
from slack_bolt.adapter.flask import SlackRequestHandler
app = App(token=SLACK_BOT_TOKEN, signing_secret=SLACK_SIGNING_SECRET)
flask_app = Flask(__name__)
handler = SlackRequestHandler(app)

In [6]:
# get_bot_user_id() #7

# Initialize the Slack app and Flask app

In [7]:
app = App(token=SLACK_BOT_TOKEN, signing_secret=SLACK_SIGNING_SECRET)
flask_app = Flask(__name__)
handler = SlackRequestHandler(app)

# Initialize the OpenAI-based language model

In [8]:
chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.7)

C:\Users\asifr\AppData\Local\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


# Create a memory buffer for conversational context

In [9]:
memory = ConversationBufferMemory()

# Create system and human prompts for Fredrick-specific conversations

In [10]:
system_template = """
You are Fredrick, an AI assistant working at the New Mexico Tax & Revenue Department. 
Your role is to assist with data-related queries, provide economic data on demand, and engage in meaningful conversations.
"""
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)

human_template = "User says: {user_input}."
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

# Create a conversational chain with memory for FRED-related queries

In [11]:
chain = LLMChain(llm=chat, prompt=chat_prompt, memory=memory)

C:\Users\asifr\AppData\Local\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


# Initialize the FRED API

In [12]:
fred = Fred(api_key=FRED_API_KEY)

# Installing & importing functions

In [13]:
!pip install functions

from functions import get_bot_user_id # 7
from functions import fetch_historical_data_from_fred # 8
from functions import get_series_id_from_keyword # 9

# Slack event handler for all messages

In [14]:
@app.event("message")
def handle_all_messages(event, say):
    user_text = event.get("text", "").lower()

    # Respond to greetings
    if user_text in ["hi", "hello", "hey"]:
        say("Hi! I'm Fredrick, your AI Assistant at New Mexico Tax & Revenue, and I specialize in managing all data requests.. How can I help you today?")
        return

    # Determine which data to fetch based on the user's request
    keyword = re.search(r"\b(\w+)\b", user_text).group(0)  # Extract the first word as the keyword
    series_id = get_series_id_from_keyword(keyword)

    if series_id:
        historical_data = fetch_historical_data_from_fred(series_id)
        say(f"Data for {keyword.upper()} from FRED:\n{historical_data}")  # Provide data with source link
    else:
        # If the series ID is not found, provide a default message with a source URL
        say("Sorry, I couldn't find any data for that query. You can check the FRED website for more information.")



# Flask endpoint for Slack events

In [15]:
@flask_app.route("/slack/events-FRED", methods=["POST"])
def slack_events():
    return handler.handle(request)

In [ ]:
# Initialize and run the Flask app
if __name__ == "__main__":
    flask_app.run(port=3000)

# ngrok start --config=ngrok-instance2.yml --all

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:3000
Press CTRL+C to quit
